In [1]:
%reload_ext autoreload
%autoreload 2

# Jos haluaa saada keskellä koodia "pretty printin" niinkuin cellin lopussa
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

import sys
import pandas as pd
import time
import timeit
sys.path.insert(0, './Moduulit')
pd.options.mode.chained_assignment = None

from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets


# Importtaus tässä muodossa
from database_connection import database_query
from outliers import find_outliers, draw_histogram
from clean_dataframe import clean_dataframe
from clean_velocities import *
from draw_node_amount import *
from draw_cluster_routes import *
from write_df_to_database import write_df_to_mariadb
from xy_diskretisointi import xy_to_grid
from reittien_erottelu import *
from isolation_forest import *
from sisään_uloskäynnit import *
from horrostila import *
from widgets import *
from plot_lapimenoajat import *
from klusterointi import *

In [2]:
df = database_query("SELECT * FROM DATA LIMIT 10000")

In [3]:
df

,ajokerta,node_id,timestamp,x,y,grid_id,distance,kesto,dayofweek,current_hour
0,2,1,2020-06-25 15:21:46,28,15,628,0.962081,6812.007,3,8
1,2,1,2020-06-25 15:21:47,28,15,628,1.028834,6812.007,3,8
2,2,1,2020-06-25 15:21:47,28,15,628,0.648460,6812.007,3,8
3,2,1,2020-06-25 15:21:47,28,15,628,0.826075,6812.007,3,8
4,2,1,2020-06-25 15:21:50,26,15,626,1.753767,6812.007,3,8
...,...,...,...,...,...,...,...,...,...,...
9995,37,1,2020-07-04 16:09:31,14,32,1294,0.284605,3109.001,5,9
9996,37,1,2020-07-04 16:09:38,14,32,1294,0.447214,3109.001,5,9
9997,37,1,2020-07-04 16:09:40,13,31,1253,0.465188,3109.001,5,9
9998,37,1,2020-07-04 16:09:40,13,31,1253,0.298329,3109.001,5,9


In [5]:
df.kesto.max()

6812.0070000000005

In [6]:
def test_function(value):
    if value < 550:
        return 0
    elif value < 3500:
        return 1
    else:
        return 2
df['type'] = df.kesto.apply(lambda x: test_function(x))
df  

,ajokerta,node_id,timestamp,x,y,grid_id,distance,kesto,dayofweek,current_hour,type
0,2,1,2020-06-25 15:21:46,28,15,628,0.962081,6812.007,3,8,2
1,2,1,2020-06-25 15:21:47,28,15,628,1.028834,6812.007,3,8,2
2,2,1,2020-06-25 15:21:47,28,15,628,0.648460,6812.007,3,8,2
3,2,1,2020-06-25 15:21:47,28,15,628,0.826075,6812.007,3,8,2
4,2,1,2020-06-25 15:21:50,26,15,626,1.753767,6812.007,3,8,2
...,...,...,...,...,...,...,...,...,...,...,...
9995,37,1,2020-07-04 16:09:31,14,32,1294,0.284605,3109.001,5,9,1
9996,37,1,2020-07-04 16:09:38,14,32,1294,0.447214,3109.001,5,9,1
9997,37,1,2020-07-04 16:09:40,13,31,1253,0.465188,3109.001,5,9,1
9998,37,1,2020-07-04 16:09:40,13,31,1253,0.298329,3109.001,5,9,1


In [9]:
df.timestamp[0].date()


datetime.date(2020, 6, 25)

In [11]:
df['time'] = df.timestamp.apply(lambda x: x.time())
df

,ajokerta,node_id,timestamp,x,y,grid_id,distance,kesto,dayofweek,current_hour,type,time
0,2,1,2020-06-25 15:21:46,28,15,628,0.962081,6812.007,3,8,2,15:21:46
1,2,1,2020-06-25 15:21:47,28,15,628,1.028834,6812.007,3,8,2,15:21:47
2,2,1,2020-06-25 15:21:47,28,15,628,0.648460,6812.007,3,8,2,15:21:47
3,2,1,2020-06-25 15:21:47,28,15,628,0.826075,6812.007,3,8,2,15:21:47
4,2,1,2020-06-25 15:21:50,26,15,626,1.753767,6812.007,3,8,2,15:21:50
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,37,1,2020-07-04 16:09:31,14,32,1294,0.284605,3109.001,5,9,1,16:09:31
9996,37,1,2020-07-04 16:09:38,14,32,1294,0.447214,3109.001,5,9,1,16:09:38
9997,37,1,2020-07-04 16:09:40,13,31,1253,0.465188,3109.001,5,9,1,16:09:40
9998,37,1,2020-07-04 16:09:40,13,31,1253,0.298329,3109.001,5,9,1,16:09:40
